# Make Figure 4

This notebooks takes all the trans genes that have positive and negative results and runs a GSEA using Reactome. It then takes a subset of genes from the top hit (Hemostasis) pathway and maps them on a large circle heat map. This heatmap focuses on coagulation and urokinase related genes. 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas.util.testing as tm
import scipy.stats
import re
import sys 
import statsmodels.stats.multitest

import gseapy as gp
from gseapy.plot import barplot, dotplot

import plot_utils as p 
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import math as math

import statsmodels.stats.multitest
from bokeh.palettes import RdBu
from bokeh.models import LinearColorMapper, ColumnDataSource, ColorBar
from bokeh.models.ranges import FactorRange
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, export_png, export_svgs
from bokeh.layouts import row


C:\Users\mtstu\AppData\Local\Temp/ipykernel_14612/2106242056.py:5: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:

def plotCircleHeatMap ( df, circle_var, color_var, x_axis, y_axis, plot_width= 1000, plot_height = 650, font_size = 12,    x_axis_lab = "no_label", y_axis_lab = "no_label", show_plot = True, save_png = "plot.png", legend_min = 1e-6, legend_med = .0001, legend_max = 0.01, show_legend = True):
  
    # circle_var designed for pvalues. Normalized by taking log 10 of values and multiplying by 5 
    #added a new column to make the plot size
    
    df["size2"] = df[circle_var].apply(lambda x: -1*(np.log(x)))
    df['size'] = (df["size2"])*3
    #find values to set color bar min/ max as 
    maxval = df[color_var].max()
    minval = df[color_var].min()
    if maxval > abs(minval):
        minval = maxval * -1 
    if maxval < abs(minval):
        maxval = minval * -1
    colors = list((RdBu[9]))
    exp_cmap = LinearColorMapper(palette=colors, low = minval, high = maxval)
    p = figure(x_range = FactorRange(), y_range = FactorRange(), plot_width= plot_width, 
               plot_height=plot_height, 
               toolbar_location=None, tools="hover")

    p.scatter(x_axis,y_axis,source=df, fill_alpha=1,  line_width=0, size="size", 
              fill_color={"field":color_var, "transform":exp_cmap})

    #p.x_range.factors = sorted(df[x_axis].unique().tolist())
    p.x_range.factors = df[x_axis].unique().tolist()
    p.y_range.factors = sorted(df[y_axis].unique().tolist(), reverse = True)
    p.xaxis.major_label_orientation = math.pi/2
    
    # font size
    p.axis.major_label_text_font_size = str(font_size)+"pt"
    p.axis.axis_label_text_font_size = str(font_size)+"pt"
    
    if (x_axis_lab != "no_label" ):
        p.xaxis.axis_label = x_axis_lab
    if (y_axis_lab != "no_label" ):   
        p.yaxis.axis_label = y_axis_lab

    bar = ColorBar(color_mapper=exp_cmap, location=(0,0))
    p.add_layout(bar, "right")
    
    if show_plot:  
        if show_legend:
            # Create Circle Legend
            circle_legend = create_circle_legend(circle_var, color_var, legend_min, legend_med, legend_max)
            circle_legend.axis.major_label_text_font_size = str(font_size - 1)+"pt" # font size
            output_notebook()
            show(row(p, circle_legend))
        else:
            output_notebook()
            show(p)
      
    if save_png != "plot.png":
        export_png(row(p, circle_legend), filename= save_png)
        
    return p
         

'''
@Param legend_min: Float. Lowest p-value to include in the legend.
@Param legend_max: Float. Highest p-value to include in the legend.
@Param color_var: Str. Used for column name (same as other df)

Returns: df to be used in creating the circle legend. 
'''

def create_circle_legend_df(color_var, legend_min, legend_med, legend_max):
    # Find middle pval
    
    #exp_min = abs(np.log10(legend_min))
    #exp_max = abs(np.log10(legend_max))
    #delta = (exp_min - exp_max) / 2 
    #exp_mid = -1 * (exp_max + delta)
    #pval2 = 1 * 10**exp_mid
    
    # Foramat scientific notation pvals as strings for y_axis labels  
    max_str = "{:.1e}".format(legend_max, '.2f')
    med_str = "{:.1e}".format(legend_med, '.2f')
    min_str = "{:.1e}".format(legend_min, '.2f')
    
    # max to min
    data = {'P_Value':  [legend_max, legend_med, legend_min],
            'y_axis': [max_str, med_str, min_str],
            'x_axis': ['', '', ''],
            color_var: [1.5, 1.5, 1.5]}

    fake_df = pd.DataFrame (data, columns = ['x_axis', 'y_axis', 'P_Value', color_var])
    
    fake_df["size2"] = fake_df['P_Value'].apply(lambda x: -1*(np.log(x)))
    fake_df['size'] = (fake_df["size2"])*3
    
    return fake_df

'''
@Param df: Dataframe. Same as df passed to plotCircleHeatMap.
@Param circle_var: Column Label. Same as passed to plotCircleHeatMap.
@Param color_var: Column Label. Same as passed to plotCircleHeatMap.
@Param x_axis: Column Label. Used on the x-axis.
@Param y_axis: Column Label. Used on the y-axis.
@Param legend_min: Float. Lowest p-value to include in the legend.
@Param legend_max: Float. Highest p-value to include in the legend.

Returns: df to be used in creating the circle legend. 
'''

def create_circle_legend(circle_var, color_var, legend_min, legend_med, legend_max,
                         x_axis = 'x_axis', y_axis = 'y_axis', 
                         plot_height = 200, plot_width = 140):
    # Get customized df
    circle_df = create_circle_legend_df(color_var, legend_min, legend_med, legend_max)
    
    maxval = circle_df[color_var].max()
    minval = circle_df[color_var].min()
    if maxval > abs(minval):
        minval = maxval * -1 
    if maxval < abs(minval):
        maxval = minval * -1
    colors = list((RdBu[9]))
    exp_cmap = LinearColorMapper(palette=colors, low = minval, high = maxval)
    
    circle = figure(x_range = FactorRange(), y_range = FactorRange(), plot_width= plot_width, 
               plot_height=plot_height, toolbar_location=None, tools="hover")

    circle.scatter(x_axis, y_axis, source = circle_df, fill_alpha=1,  line_width=0, size="size", 
              fill_color={"field":color_var, "transform":exp_cmap})
    
    circle.x_range.factors = sorted(circle_df[x_axis].unique().tolist())
    circle.y_range.factors = circle_df[y_axis].unique().tolist() # plots in reverse order of circle_df (max to min)
    circle.xaxis.major_label_orientation = math.pi/2
    
    circle.xaxis.axis_label = 'FDR p-value'
    
    return circle



# Step 1: Find Trans proteins with opposite effects in different cancers 

Load df with all of the genes that are FDR significant. This dataframe was made in the Make_Supplemental_Tables notebook. See
https://github.com/PayneLab/WhenMutationsDontMatter/blob/master/EGFR/Make_Tables/Make_Supplemental_Tables.ipynb

In [3]:
FDR_sig = pd.read_csv("csv_files/Supplemental_Table_EGFR_sig_only.csv")
FDR_sig = FDR_sig.set_index("Comparison")



In [4]:
FDR_sig.max(axis=1)
FDR_sig.min(axis = 1)

Comparison
PHLDA1      9.649849e-19
CDH4        4.744022e-10
GRB2       -5.890078e-01
SOCS2       2.006861e-06
PHLDA3      2.351012e-14
                ...     
VPS37B     -2.461344e-01
SCN10A     -3.648703e-01
PCYT1A      4.976210e-02
NCBP1       4.976210e-02
IGHV3-43   -2.451771e-01
Length: 6376, dtype: float64

In [5]:
def HasPosNeg(row):
    hasPos = False
    hasNeg= False

    for item in row:
        if pd.isnull(item):
            continue
        if item < 0:
            hasNeg = True
        if item > 0:
            hasPos = True
            
    if hasPos & hasNeg:
        return True
    return False

Subset data frame to include only trans genes that have opposite effects in different cancers by using apply function

In [6]:
col = ["Correlation_GBM","Correlation_ccRCC","Correlation_BR","Correlation_LUAD","Correlation_HNSCC","Correlation_LSCC"]
FDR_corr = FDR_sig[col]
FDR_corr["Pos_Neg"] = FDR_corr.apply(HasPosNeg, axis = 1)
FDR_corr_True = FDR_corr[FDR_corr['Pos_Neg']==True]
FDR_corr_True.head(20)

C:\Users\mtstu\AppData\Local\Temp/ipykernel_14612/4294753839.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FDR_corr["Pos_Neg"] = FDR_corr.apply(HasPosNeg, axis = 1)


,Correlation_GBM,Correlation_ccRCC,Correlation_BR,Correlation_LUAD,Correlation_HNSCC,Correlation_LSCC,Pos_Neg
Comparison,,,,,,,
CTSC,-0.548303,NaN,0.314172,0.282360,NaN,NaN,True
SHC1,-0.521169,0.361057,NaN,NaN,NaN,NaN,True
HSD17B11,-0.521630,NaN,0.400780,NaN,NaN,NaN,True
CTSB,-0.514005,NaN,0.377784,NaN,NaN,NaN,True
SCPEP1,-0.509046,-0.394109,0.388977,NaN,NaN,NaN,True
DAB2,-0.500648,NaN,0.350933,NaN,NaN,NaN,True
OSTF1,-0.500063,-0.456002,0.281638,NaN,NaN,NaN,True
PLA2G15,-0.498087,NaN,0.289403,NaN,NaN,NaN,True
SUB1,0.498468,-0.289319,NaN,NaN,NaN,NaN,True


In [7]:
def Pvalue_sig(row):
    numSig = 0

    for item in row:
        if pd.isnull(item):
            continue
        if item < 0.05:
            numSig += 1
            
    return numSig

In [8]:
df = FDR_corr_True.drop("Pos_Neg",axis = 1)
diff = df.max(axis=1) - df.min(axis = 1)
diff.sort_values(ascending = False).head(20)

Comparison
CELSR1      1.025849
ARHGAP10    1.003604
MRTFB       0.941977
CTNND2      0.933537
CYRIA       0.931863
TES         0.928830
CRYBG3      0.927166
HSD17B11    0.922410
SLC9A3R1    0.915562
ARFGEF3     0.911232
JPT2        0.908296
UPP1        0.905642
SCPEP1      0.898023
FAM89A      0.896054
PLA2G4A     0.894060
RARA        0.893161
VDR         0.892795
CTSB        0.891790
CA12        0.885282
SHC1        0.882226
dtype: float64

In [9]:
abs_val = FDR_corr_True.abs()
abs_val.sum(1).sort_values(ascending = False).head(20)

Comparison
PFKP       2.794904
CELSR1     2.751942
CRYBG1     2.723200
FERMT1     2.667788
UPP1       2.637102
CD109      2.634134
GMFG       2.620877
FAM89A     2.616274
P4HA2      2.558186
JPT2       2.501748
ITGB1      2.494947
RBM47      2.494912
CDK6       2.484177
LCP1       2.476295
CA12       2.469833
IKBIP_1    2.414612
TNK1       2.405448
GGCX       2.403291
WIPF1      2.402320
BAG2       2.391618
dtype: float64

The manuscript mentions 945 of trans proteins that opposite effects in different cancers. Here is the derivation of that number 

In [10]:
pos_neg_prot = FDR_corr_True.index.tolist()
pos_neg_prot
#print("Total number of trans proteins with opposite effects in different cancers is " + str(len(pos_neg_prot)))

['CTSC',
 'SHC1',
 'HSD17B11',
 'CTSB',
 'SCPEP1',
 'DAB2',
 'OSTF1',
 'PLA2G15',
 'SUB1',
 'CTSZ',
 'RIN2',
 'PPP2R3A',
 'MTARC2',
 'SIPA1L3',
 'CTSL',
 'ARSB',
 'CHDH',
 'DOCK11',
 'SVIL',
 'GSTO1',
 'FCGR2A',
 'TES',
 'NIBAN2',
 'SNAPIN',
 'GBA',
 'COL1A1',
 'COL1A2',
 'PHLDB2',
 'ALDH7A1',
 'CTNND2',
 'PARVG',
 'RAB3IL1',
 'CDKN2C',
 'EML4',
 'ENG',
 'NPC2',
 'TMEM87B',
 'EHBP1L1',
 'ITGB1',
 'PWWP3A',
 'KYNU',
 'PTER',
 'TNS3',
 'PCMTD2',
 'RGS12',
 'DOK3',
 'VAMP8',
 'TBXAS1',
 'LPXN',
 'ACSS1',
 'EMB',
 'ACSL1',
 'APC',
 'FHOD1',
 'OTULIN',
 'CAPZA1',
 'ARHGAP10',
 'LIG4',
 'FHL2',
 'MFSD4B',
 'TWF2',
 'TNFAIP8',
 'ARPC3',
 'LUZP1',
 'FBXL17',
 'GMFG',
 'PLCG2',
 'PPM1F',
 'RALGAPB',
 'SYK',
 'WIPF1',
 'RASSF4',
 'DAPK3',
 'SULF1',
 'CTSS',
 'NOVA1',
 'DNAJC21',
 'HCK',
 'CHST3',
 'PAPSS2',
 'LETM1',
 'TPD52L2',
 'THEMIS2',
 'DIAPH2',
 'RASAL2',
 'PLIN2',
 'STAT6',
 'SDR39U1',
 'AMPD3',
 'CDK6',
 'LAIR1',
 'MANBA',
 'ARPC1B',
 'ILK',
 'CD14',
 'PIK3AP1',
 'EHD4',
 'CST3',
 'MYOF

# Run GSEA

In [11]:
pos_neg_enr = gp.enrichr(gene_list = pos_neg_prot, description='Tumor_partition', gene_sets='Reactome_2016')
pos_neg_enr.res2d.head(10)

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes
0,Reactome_2016,Fcgamma receptor (FCGR) dependent phagocytosis...,22/120,1.312739e-09,9.550146e-07,0,0,5.655427,115.659995,LYN;ACTR3;HSP90AA1;AHCYL1;SYK;WIPF1;WIPF2;ARPC...
1,Reactome_2016,Innate Immune System Homo sapiens R-HSA-168249,68/807,1.981358e-09,9.550146e-07,0,0,2.384937,47.792905,AHCYL1;WIPF1;ARPC1B;WIPF2;ITGB2;ARRB2;C8A;CLU;...
2,Reactome_2016,"Platelet activation, signaling and aggregation...",30/253,6.528625e-08,2.097865e-05,0,0,3.402817,56.297855,SHC1;DGKA;PLEK;F13A1;ARRB2;RASGRP2;CLU;ISLR;GN...
3,Reactome_2016,Regulation of actin dynamics for phagocytic cu...,17/97,1.941177e-07,4.678236e-05,0,0,5.323348,82.271278,ACTR3;HSP90AA1;SYK;WIPF1;WIPF2;ARPC1B;ARPC4;AR...
4,Reactome_2016,Hemostasis Homo sapiens R-HSA-109582,47/552,4.987805e-07,9.616488e-05,0,0,2.372887,34.433199,ITGB1;DOCK5;CD84;SHC1;DGKA;ITGB2;PLEK;F13A1;AR...
5,Reactome_2016,Collagen formation Homo sapiens R-HSA-1474290,15/85,9.218392e-07,1.481088e-04,0,0,5.356863,74.443764,COL15A1;CRTAP;COL11A1;COL12A1;CTSS;LOXL2;COL1A...
6,Reactome_2016,Metabolism Homo sapiens R-HSA-1430728,115/1908,1.497695e-06,2.062540e-04,0,0,1.678190,22.507180,RPL4;ALDH1L1;FAAH;AHCYL1;PITPNB;ALDH1L2;ENO3;R...
7,Reactome_2016,Immune System Homo sapiens R-HSA-168256,97/1547,2.112067e-06,2.452168e-04,0,0,1.737843,22.709864,AHCYL1;NCF1;WIPF1;WIPF2;NCF4;CLU;NPEPPS;RPS6KA...
8,Reactome_2016,Extracellular matrix organization Homo sapiens...,29/283,2.289368e-06,2.452168e-04,0,0,2.879382,37.395210,DDR1;ITGB1;COL15A1;ITGB5;COL11A1;COL12A1;ITGB2...
9,Reactome_2016,Terminal pathway of complement Homo sapiens R-...,5/8,4.551068e-06,4.387229e-04,0,0,41.271478,507.645312,C8G;C6;C9;C8A;CLU


In [12]:
pos_neg_df = pos_neg_enr.res2d
fcgr = pos_neg_df.iloc[2,9]
fcgr = fcgr.split(';')


In [13]:
#get just the clotting cascade genes and add urokinase genes 
pos_neg_df = pos_neg_enr.res2d
coag = pos_neg_df.iloc[2,9]
coag = coag.split(';')
upa = ["F3","PLAUR","PLAU","PLG","MMP9","MMP12","SERPINE1"]
coag_upa =  coag + upa
len(coag_upa)

37

# Step 3 Make Data frame for Figure 4

In [14]:
#Get append version of the df with all cancer type, fdr sig trans results
df_FDR_append = pd.read_csv("csv_files/sig_prot_heatmap_EGFR.csv")
 
#coag_upa =  ["F2""F3","F9","F10","F11","F13A1","PLAUR","PLAU","PLG","MMP9","MMP12","SERPINE1"]
#subset dataframe to include genes only desired for figure 
df_FDR_append= df_FDR_append[df_FDR_append.Comparison.isin(fcgr)]
df_FDR_append

,Comparison,Correlation,P_Value,Cancer
12,SHC1,-0.521169,0.000022,GBM
243,PLCG2,-0.390149,0.002829,GBM
248,SYK,-0.389025,0.002931,GBM
402,PFN1,-0.361363,0.006317,GBM
432,FCER1G,-0.357066,0.007069,GBM
...,...,...,...,...
8390,PLA2G4A,-0.324576,0.013817,ccRCC
8489,TMSB4X,-0.310140,0.019902,ccRCC
8559,PLCG2,-0.302984,0.023092,ccRCC
8634,SYK,-0.293638,0.028987,ccRCC


Set add new column to be unique index and order the new index. This way genes will be grouped by coagulation factors, regulators, and urokinase genes.

In [15]:

df_FDR_append["Index"] = df_FDR_append["Comparison"] + " " + df_FDR_append["Cancer"]
'''
df_FDR_append = df_FDR_append.set_index("Index")
df_ordered = df_FDR_append.reindex(["F2 GBM","F3 GBM","F9 GBM","F10 GBM","F11 GBM","F13A1 GBM","F13B GBM","KLKB1 GBM","VWF CO","FGA GBM","FGB GBM","FGG GBM","SERPINC1 GBM", "SERPIND1 GBM","SERPING1 GBM","A2M GBM","PROS1 GBM","PROC OV","PROCR GBM","THBD GBM","KNG1 GBM","PLAUR GBM","PLAU GBM","PLG GBM","MMP9 BR","MMP12 BR","SERPINE1 GBM",
                                "F2 BR","F9 BR","F10 BR","F11 BR","F13A1 BR","F13B BR","FGA BR","FGB BR","FGG BR", "SERPIND1 BR","SERPING1 BR","A2M BR","PROS1 BR","PROCR BR","KLKB1 BR", "PLAUR BR","PLAU BR","PLG BR","SERPINE1 BR",
                               "VWF HNSCC","THBD HNSCC","PLAUR HNSCC","PLAU HNSCC","SERPINE1 HNSCC",
                               "F9 LUAD","F13A1 LUAD", "F13B LUAD", "SERPIND1 LUAD","PROS1 LUAD","PROC LUAD","VWF LUAD",
                                "PROCR ccRCC",
                                "SERPIND1 OV","PROC OV",
                               "F3 CO","SERPINC1 CO", "SERPIND1 CO","A2M CO","KNG1 CO","KLKB1 CO"])
'''
df_FDR_append


,Comparison,Correlation,P_Value,Cancer,Index
12,SHC1,-0.521169,0.000022,GBM,SHC1 GBM
243,PLCG2,-0.390149,0.002829,GBM,PLCG2 GBM
248,SYK,-0.389025,0.002931,GBM,SYK GBM
402,PFN1,-0.361363,0.006317,GBM,PFN1 GBM
432,FCER1G,-0.357066,0.007069,GBM,FCER1G GBM
...,...,...,...,...,...
8390,PLA2G4A,-0.324576,0.013817,ccRCC,PLA2G4A ccRCC
8489,TMSB4X,-0.310140,0.019902,ccRCC,TMSB4X ccRCC
8559,PLCG2,-0.302984,0.023092,ccRCC,PLCG2 ccRCC
8634,SYK,-0.293638,0.028987,ccRCC,SYK ccRCC


# Step 4: Plot Figure 4

In [16]:
legend_min = df_FDR_append["P_Value"].min()
#Make plot using plot utils
p.plotCircleHeatMap(df_FDR_append, circle_var = "P_Value",color_var = "Correlation", x_axis = "Comparison", y_axis = "Cancer", plot_width= 700, plot_height = 500, legend_min = legend_min, legend_max = 0.05, font_size = 10, show_legend = True)

Loading BokehJS ...

Figure(id='1005', ...)

# Check if blanks are due to no data 

The follow code chunks show that the following cancers/genes don't have data: colon THBD, Kidney MMP12, and Ovarian MMP12. (As mentioned in EGFR Figure 2 legend)

In [18]:
#Get append version of the df with all proteins 
df_all_prot_append = pd.read_csv("csv_files/all_prot_heatmap_EGFR.csv")
df_all_prot_append 


,Comparison,Correlation,P_Value,Cancer
0,EGFR,1.000000,0.000000e+00,GBM
1,PHLDA1,0.789551,9.649849e-19,GBM
2,CDH4,0.656331,4.744022e-10,GBM
3,GRB2,-0.589008,3.310325e-07,GBM
4,SOCS2,0.565367,2.006861e-06,GBM
...,...,...,...,...
67276,RPS29,0.000111,9.997098e-01,ccRCC
67277,YBX1,0.000072,9.997098e-01,ccRCC
67278,UBL7,0.000063,9.997098e-01,ccRCC
67279,HEATR3,0.000043,9.997470e-01,ccRCC


In [19]:
#subset dataframe to include genes only desired for figure 
df_all_comp_coag = df_all_prot_append[df_all_prot_append.Comparison.isin(coag_upa)]
print("Number of rows in data frame " + str(len(df_all_comp_coag)))


Number of rows in data frame 218


Our figure includes 27 genes for 8 cancers. If all data was present there would be 216 rows. However, the data frame only has 212 rows. 4 genes are missing. 

In [20]:
def find_missing_genes(test_list, full_list):
    for gene in full_list:
        if (gene not in test_list):
            print(gene)
        

In [21]:
#Get list of genes for colon, kidney, and ovarian
colon = df_all_prot_append[df_all_prot_append["Cancer"] == "CO"]
colon_list = colon.Comparison.to_list()

Kidney = df_all_prot_append[df_all_prot_append["Cancer"] == "ccRCC"]
Kidney_list = Kidney.Comparison.to_list()

Ovarian = df_all_prot_append[df_all_prot_append["Cancer"] == "OV"]
Ovarian_list = Ovarian.Comparison.to_list()

In [22]:
#Show the 3 missing genes 
print("Ovarian missing genes: ")
find_missing_genes(Ovarian_list, coag_upa)
print("Kidney missing genes: ")
find_missing_genes(Kidney_list, coag_upa)
print("Colon missing genes: ")
find_missing_genes(colon_list, coag_upa)

Ovarian missing genes: 
SHC1
DGKA
PLEK
F13A1
ARRB2
RASGRP2
CLU
ISLR
GNG2
TMSB4X
GNG7
PLCG2
FYN
A2M
LYN
SRGN
FGB
FCER1G
TGFB1
SYK
PLA2G4A
MAPK14
VAV2
RHOB
DGKQ
PECAM1
SERPING1
TLN1
PFN1
VCL
F3
PLAUR
PLAU
PLG
MMP9
MMP12
SERPINE1
Kidney missing genes: 
SRGN
MMP12
Colon missing genes: 
SHC1
DGKA
PLEK
F13A1
ARRB2
RASGRP2
CLU
ISLR
GNG2
TMSB4X
GNG7
PLCG2
FYN
A2M
LYN
SRGN
FGB
FCER1G
TGFB1
SYK
PLA2G4A
MAPK14
VAV2
RHOB
DGKQ
PECAM1
SERPING1
TLN1
PFN1
VCL
F3
PLAUR
PLAU
PLG
MMP9
MMP12
SERPINE1
